In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

import pandas as pd

In [2]:
movies=pd.read_csv('movies_cleaned.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36387 entries, 0 to 36386
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  36387 non-null  int64  
 1   name                36387 non-null  object 
 2   date                36387 non-null  float64
 3   description         36387 non-null  object 
 4   minute              36387 non-null  float64
 5   rating              36309 non-null  float64
 6   actor               35566 non-null  object 
 7   director            36279 non-null  object 
 8   genre               36223 non-null  object 
 9   language            35407 non-null  object 
 10  poster              36385 non-null  object 
 11  theme               36387 non-null  object 
 12  tagged_description  36387 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 3.6+ MB


In [3]:
movies['tagged_description'].to_csv('tagged_description.txt',index=False,header=False,sep='\n')

In [4]:
raw_documents=TextLoader('tagged_description.txt',encoding='utf-8').load()
text_splitters=CharacterTextSplitter(chunk_size=1,chunk_overlap=0,separator='\n')
documents=text_splitters.split_documents(raw_documents)

Created a chunk of size 237, which is longer than the specified 1
Created a chunk of size 172, which is longer than the specified 1
Created a chunk of size 187, which is longer than the specified 1
Created a chunk of size 307, which is longer than the specified 1
Created a chunk of size 370, which is longer than the specified 1
Created a chunk of size 108, which is longer than the specified 1
Created a chunk of size 206, which is longer than the specified 1
Created a chunk of size 177, which is longer than the specified 1
Created a chunk of size 451, which is longer than the specified 1
Created a chunk of size 247, which is longer than the specified 1
Created a chunk of size 470, which is longer than the specified 1
Created a chunk of size 138, which is longer than the specified 1
Created a chunk of size 170, which is longer than the specified 1
Created a chunk of size 361, which is longer than the specified 1
Created a chunk of size 377, which is longer than the specified 1
Created a 

In [5]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='1000001 Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.')

In [10]:
from langchain_core.documents import Document

documents_with_metadata = []
for i, doc in enumerate(documents):
    movie_idx = i % len(movies)
    row = movies.iloc[movie_idx]
    
    new_doc = Document(
        page_content=doc.page_content,
        metadata={
            'name': row['name'],
            'date': row['date'],
            'actor': row['actor'],
            'director': row['director'],
            'language': row['language'],
        }
    )
    documents_with_metadata.append(new_doc)


In [44]:
documents_with_metadata[:10]

[Document(metadata={'name': 'Barbie', 'date': np.float64(2023.0), 'actor': 'Margot Robbie, Ryan Gosling, America Ferrera, Ariana Greenblatt, Issa Rae', 'director': 'Greta Gerwig', 'language': 'English'}, page_content='1000001 Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chance to go to the real world, they soon discover the joys and perils of living among humans.'),
 Document(metadata={'name': 'Parasite', 'date': np.float64(2019.0), 'actor': 'Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong, Choi Woo-shik, Park So-dam', 'director': 'Bong Joon-ho', 'language': 'Korean, English, German, Korean'}, page_content="1000002 All unemployed, Ki-taek's family takes peculiar interest in the wealthy and glamorous Parks for their livelihood until they get entangled in an unexpected incident."),
 Document(metadata={'name': 'Everything Everywhere All at Once', 'date': np.float64(2022.0), 'actor': 'Michelle Yeoh, Ke Huy 

In [11]:
embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
db_movies=Chroma.from_documents(documents_with_metadata,
                                embedding=embedding_model,
                                persist_directory='chroma_db')

In [53]:
query='barbie'
docs=db_movies.similarity_search(query,k=5)
docs

[Document(id='eb8c15e1-7a34-4a91-b8a8-6be5bd7e6d48', metadata={'actor': 'Kelly Sheridan, Britt Irvin, Michael Kopsa, Rebecca Husain, Kira Tozer', 'director': 'Ezekiel Norton', 'date': 2015.0, 'language': 'English', 'name': 'Barbie in Princess Power'}, page_content='1015402 Barbie is Kara - a modern-day princess with a normal life. Kissed by a butterfly which gives her superpowers and allows her to become a Super Sparkle, she is ready to save the kingdom from evil - were it not for her jealous cousin who is also kissed by the butterfly and becomes her rival and nemesis. Watch as Super Sparkle and Dark Sparkle learn that together they can become a great team for good once they learn the power of friendship.'),
 Document(id='0da9a7dd-a0f5-4d97-9638-96905b13ff6c', metadata={'director': 'William Lau', 'name': 'Barbie: A Fashion Fairytale', 'language': 'English', 'date': 2010.0, 'actor': 'Diana Kaarina, Adrian Petriw, Tabitha St. Germain, Patricia Drake, Alexa Devine'}, page_content="1005177

In [153]:
movies[movies['id']==int(docs[0].page_content.split()[0].strip())]

,id,name,date,description,minute,rating,actor,director,genre,language,poster,theme,tagged_description
15340,1015402,Barbie in Princess Power,2015.0,Barbie is Kara - a modern-day princess with a ...,74.0,2.39,"Kelly Sheridan, Britt Irvin, Michael Kopsa, Re...",Ezekiel Norton,Animation,English,https://a.ltrbxd.com/resized/film-poster/2/4/3...,Barbie is Kara - a modern-day princess with a ...,1015402 Barbie is Kara - a modern-day princess...


In [190]:
def enhanced_similarity_search(query, vector_store=db_movies, movies_df=movies, top_k=10):
    metadata_matches = []
    
    # Check if query matches any movie names
    name_matches = movies_df[movies_df['name'].str.contains(query, case=False, na=False)]
    for _, movie in name_matches.iterrows():
        docs = vector_store.similarity_search(
            query, 
            k=top_k,
            filter={"name": movie['name']}
        )
        metadata_matches.extend(docs)
    
    # Check if query matches any directors
    director_matches = movies_df[movies_df['director'].str.contains(query, case=False, na=False)]
    for _, movie in director_matches.iterrows():
        docs = vector_store.similarity_search(
            query, 
            k=top_k,
            filter={"director": movie['director']}
        )
        metadata_matches.extend(docs)
    
    # Check if query matches any actors
    actor_matches = movies_df[movies_df['actor'].str.contains(query, case=False, na=False)]
    for _, movie in actor_matches.iterrows():
        docs = vector_store.similarity_search(
            query, 
            k=top_k,
            filter={"actor": movie['actor']}
        )
        metadata_matches.extend(docs)
    
    # Check if query matches any dates
    date_matches = movies_df[movies_df['date'].astype(str).str.contains(query, na=False)]
    for _, movie in date_matches.iterrows():
        docs = vector_store.similarity_search(
            query, 
            k=top_k,
            filter={"date": movie['date']}
        )
        metadata_matches.extend(docs)
    
    # Remove duplicates
    seen_ids = set()
    unique_metadata_matches = []
    for doc in metadata_matches:
        doc_id = f"{doc.metadata['name']}_{doc.page_content[:7]}"
        if doc_id not in seen_ids:
            seen_ids.add(doc_id)
            unique_metadata_matches.append(doc)
    
    movies_list = []
    
    # If we have metadata matches, use them
    if unique_metadata_matches:
        for doc in unique_metadata_matches:
            try:
                movie_id = int(doc.page_content.strip('"').split()[0])
                movies_list.append(movie_id)
            except (ValueError, IndexError, AttributeError) as e:
                print(f"Error extracting movie ID from document: {e}")
                continue
    else:
        regular_results = vector_store.similarity_search(query, k=top_k)
        for doc in regular_results:
                movie_id = int(doc.page_content.strip('"').split()[0])
                movies_list.append(movie_id)
    
    return movies[movies['id'].isin(movies_list)].head(top_k)

In [194]:
enhanced_similarity_search('a long search')

,id,name,date,description,minute,rating,actor,director,genre,language,poster,theme,tagged_description
3371,1003373,Once Upon a Time in Anatolia,2011.0,In the rural area around the Anatolian town of...,163.0,4.13,"Muhammet Uzuner, Yılmaz Erdoğan, Taner Birsel,...",Nuri Bilge Ceylan,"Crime, Drama",Turkish,https://a.ltrbxd.com/resized/film-poster/6/1/0...,Thrillers and murder mysteries.Humanity and th...,1003373 In the rural area around the Anatolian...
10484,1010513,Mystery Team,2009.0,A group of former Encyclopedia Brown-style chi...,97.0,3.34,"Donald Glover, Aubrey Plaza, Dominic Dierkes, ...",Dan Eckman,"Comedy, Crime, Mystery, Thriller",English,https://a.ltrbxd.com/resized/sm/upload/b1/wr/b...,"Crude humor and satire.Gags, jokes, and slapst...",1010513 A group of former Encyclopedia Brown-s...
17599,1017683,The Pursuit of Love,2021.0,"Longing for love, obsessed with sex, Linda is ...",177.0,3.45,"Lily James, Emily Beecham, Andrew Scott, Domin...",Emily Mortimer,Drama,English,https://a.ltrbxd.com/resized/film-poster/7/1/5...,Moving relationship stories.Touching and senti...,"1017683 Longing for love, obsessed with sex, L..."
21806,1021926,Jagga Jasoos,2017.0,A gifted teenage detective searches for his mi...,180.0,3.00,"Ranbir Kapoor, Katrina Kaif, Sayani Gupta, Sas...",Anurag Basu,"Romance, Comedy, Adventure, Mystery",Hindi,https://a.ltrbxd.com/resized/film-poster/3/3/5...,Song and dance.Bollywood emotional dramas.Dazz...,1021926 A gifted teenage detective searches fo...
25214,1025368,Midnight Madness,1980.0,A genius grad student organizes an all-night t...,112.0,3.15,"David Naughton, Debra Clinger, Eddie Deezen, M...","Michael Nankin, David Wechter","Family, Comedy","English, English, Spanish",https://a.ltrbxd.com/resized/film-poster/4/0/6...,Crude humor and satire.Teen school antics and ...,1025368 A genius grad student organizes an all...
26253,1026419,The Sum of Us,1994.0,A widowed father has to deal with two complex ...,100.0,3.53,"Jack Thompson, Russell Crowe, John Polson, Deb...","Kevin Dowling, Geoff Burton","Comedy, Drama, Romance",English,https://a.ltrbxd.com/resized/film-poster/2/7/7...,Relationship comedy.Moving relationship storie...,1026419 A widowed father has to deal with two ...
28687,1028879,Suspect X,2008.0,A male corpse is discovered with a smashed fac...,128.0,3.62,"Masaharu Fukuyama, Ko Shibasaki, Kazuki Kitamu...",Hiroshi Nishitani,"Drama, Crime, Mystery",Japanese,https://a.ltrbxd.com/resized/film-poster/2/2/7...,A male corpse is discovered with a smashed fac...,1028879 A male corpse is discovered with a sma...
31957,1032186,Catcalls,2017.0,A man cruises around late at night looking for...,9.0,3.08,"Martin O'Sullivan, Edel Murphy, Cesca Saunders...",Kate Dolan,Horror,English,https://a.ltrbxd.com/resized/film-poster/4/3/2...,A man cruises around late at night looking for...,1032186 A man cruises around late at night loo...
32442,1032676,Brutal,2017.0,A savage murderer is on the prowl in Japan. On...,66.0,2.73,"Butch, Ayano, Takashi Nishina, Asami, Katrina ...",Takashi Hirose,Horror,"Japanese, Japanese, English",https://a.ltrbxd.com/resized/film-poster/4/1/9...,A savage murderer is on the prowl in Japan. On...,1032676 A savage murderer is on the prowl in J...
34965,1035230,Believer 2,2023.0,A determined detective continues his search fo...,116.0,2.58,"Cho Jin-woong, Cha Seung-won, Han Hyo-joo, Oh ...",Baik,Thriller,"Korean, Chinese, Korean",https://a.ltrbxd.com/resized/film-poster/8/6/2...,A determined detective continues his search fo...,1035230 A determined detective continues his s...
